In [84]:
import pandas as pd
import numpy as np

In [85]:
# Read file in
df = pd.read_csv('./skus_orders_day.csv')
# df = pd.read_csv('./100kskus.csv')

# TODO Delete this line when done!!!
# df = df.head(10000)
df.head()

,SKU,Day,Order#,DIVISION,sum
0,3857515,2017-12-24,E19309493,FTW,1
1,3857516,2017-12-19,E19160272,FTW,1
2,3857518,2017-12-19,E19179588,FTW,1
3,3857519,2017-12-19,E19179156,FTW,1
4,3857565,2017-12-20,E19211037,FTW,1


In [86]:
####
n = 200
top_n_skus = df.groupby('SKU')[['sum']].sum().nlargest(n, 'sum')
sample_df = pd.merge(df, top_n_skus, how='inner', on=['SKU']).drop(columns=['sum_y']).rename(columns={'sum_x':'sum'})
#####


select_df = sample_df[['SKU','Order#']]
all_select_df = df[['SKU', 'Order#']]

# #-Containing Orders

# NOTE: By joining itself, we are only seeing combos of top 10 with top 10. However, you can also do top 10 with the
# rest of the orig dataset, to see how top 10 is paired with the rest of SKUs

# joined = pd.merge(select_df, select_df, how='inner', on='Order#', suffixes=['_1','_2'])
joined = pd.merge(select_df, all_select_df, how='inner', on='Order#', suffixes=['_1','_2'])

# # BELOW FOR TESTING
# s = joined[joined['Order#'] == 'E11863102']
# s[s['SKU_1']!=s['SKU_2']].groupby(['SKU_1','SKU_2']).count()
# ###

# Omit whenever SKU1 and SKU2 are same SKU
# Group by [SKU1,SKU2] to see how many orders contain them
cont_ord_df = joined[joined['SKU_1']!=joined['SKU_2']].groupby(['SKU_1','SKU_2']).count().rename(columns={"Order#": "#-Containing Orders"}).reset_index()

In [87]:
df.loc[0:500000,].to_csv('./yan.csv',index=False)


In [88]:
# #-Completing Orders
grouped = select_df.groupby('Order#').filter(lambda x: len(x)==2)

merged = pd.merge(grouped, grouped, how='inner', on='Order#', suffixes=['_1','_2'])
comp_ord_df = merged[merged['SKU_1']!=merged['SKU_2']].groupby(['SKU_1','SKU_2']).count().rename(columns={"Order#": "#-Completing Orders"}).reset_index()

In [89]:
paired_skus_df = pd.merge(cont_ord_df, comp_ord_df, how='left', on=['SKU_1','SKU_2']).fillna(0)

# Pick top 10 selling pairs of SKUs (by number of orders)
top_paired_skus = paired_skus_df.nlargest(1000*2,"#-Containing Orders")

top_paired_skus_sum_1_only = pd.merge(
    top_paired_skus, 
    df.groupby('SKU')[['sum']].sum().reset_index().rename(columns={'SKU':'SKU_1'}), 
    how='left', on=['SKU_1']).rename(columns={'sum':'SKU_1_Sum'})

edges = pd.merge(
    top_paired_skus_sum_1_only, 
    df.groupby('SKU')[['sum']].sum().reset_index().rename(columns={'SKU':'SKU_2'}), 
    how='left', on=['SKU_2']).rename(columns={'sum':'SKU_2_Sum'})

edges.head()

,SKU_1,SKU_2,#-Containing Orders,#-Completing Orders,SKU_1_Sum,SKU_2_Sum
0,9891853,9891854,1880,798.0,14301,16177
1,9891854,9891853,1880,798.0,16177,14301
2,9891854,9891855,1712,595.0,16177,13164
3,9891855,9891854,1712,595.0,13164,16177
4,9891854,9891856,1554,495.0,16177,15401


In [90]:
pairs=set(zip(edges.SKU_1,edges.SKU_2,edges["#-Containing Orders"],edges["#-Completing Orders"],edges["SKU_1_Sum"],edges["SKU_2_Sum"]))


In [91]:
pairs=set(zip(edges.SKU_1,edges.SKU_2))
u_pairs=list(set((a,b) if a<=b else (b,a) for a,b in pairs) )
u_pairs_df = pd.DataFrame(u_pairs,columns=['SKU_1','SKU_2'])
u_edges=pd.merge(edges, u_pairs_df, how='inner', on=['SKU_1','SKU_2'])
u_edges.head()

,SKU_1,SKU_2,#-Containing Orders,#-Completing Orders,SKU_1_Sum,SKU_2_Sum
0,9891853,9891854,1880,798.0,14301,16177
1,9891854,9891855,1712,595.0,16177,13164
2,9891854,9891856,1554,495.0,16177,15401
3,9891853,9891855,1501,548.0,14301,13164
4,9891856,9891857,1481,640.0,15401,11385


In [97]:
# From the top selling pairs, grab the unique SKUs and their totals
tpss_sku1 = u_edges[['SKU_1','SKU_1_Sum']].rename(columns={'SKU_1':'SKU', "SKU_1_Sum":"Sum"})
tpss_sku2 = u_edges[['SKU_2', 'SKU_2_Sum']].rename(columns={'SKU_2':'SKU', "SKU_2_Sum":"Sum"})
nodes = pd.concat([tpss_sku1, tpss_sku2], axis=0).drop_duplicates()
nodes.head()

,SKU,Sum
0,9891853,14301
1,9891854,16177
4,9891856,15401
5,9891855,13164
8,9908177,8392


In [98]:
# Add degrees columns to Node dataframe
temp = pd.DataFrame(data={'SKU':u_edges['SKU_1'].append(u_edges['SKU_2'])})
node_degrees = pd.DataFrame(data={'Degrees':temp.groupby('SKU')['SKU'].count()}).reset_index()

nodes = pd.merge(nodes, node_degrees, how='left', on=['SKU'])

In [99]:
# Add DIVISION column to Node dataframe
# Combine Divisions FTW and APP as one Division 'APP/FTW', since some SKUs are labels as both
t = df[['SKU','DIVISION']]
t['DIVISION'] = np.where((t['DIVISION']=='APP') | (t['DIVISION']=='FTW') , 'APP/FTW', t['DIVISION'])
sku_div_df = t.drop_duplicates(['SKU','DIVISION'])

nodes = pd.merge(nodes, sku_div_df, how='left', on=['SKU'])


/Users/DaveSomo/anaconda3/envs/deeplearning/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [100]:
u_edges.iloc[:,0:3].to_csv('./edges.csv',index=False)
nodes.to_csv('./nodes.csv',index=False)

In [101]:
# Strength Bin Distribution
n_bins=100
sorted_degrees = sorted(nodes['Sum'])


len(sorted_degrees)
space=len(sorted_degrees)/n_bins

dist=[]
for i in range(n_bins):
    
    l_idx = int(i * space)
    r_idx = int((i+1) * space)
    curr_bin = np.array(sorted_degrees[l_idx:r_idx])    
    dist.append(np.array([i, np.mean(curr_bin), len(curr_bin)]))
dist = np.array(dist)

dist_strength = pd.DataFrame(data={'Bin':dist[:,0], 'BinMean':dist[:,1], 'Count':dist[:,2]  })
dist_strength.head()

,Bin,BinMean,Count
0,0.0,108.600000,5.0
1,1.0,140.833333,6.0
2,2.0,166.666667,6.0
3,3.0,182.666667,6.0
4,4.0,204.800000,5.0


In [102]:
nodes.head()

,SKU,Sum,Degrees,DIVISION
0,9891853,14301,28,APP/FTW
1,9891854,16177,31,APP/FTW
2,9891856,15401,28,APP/FTW
3,9891855,13164,26,APP/FTW
4,9908177,8392,16,APP/FTW


In [103]:
# Degrees Bin Distribution
n_bins=100
sorted_degrees = sorted(nodes['Degrees'])


len(sorted_degrees)
space=len(sorted_degrees)/n_bins

dist=[]
for i in range(n_bins):
    
    l_idx = int(i * space)
    r_idx = int((i+1) * space)
    curr_bin = np.array(sorted_degrees[l_idx:r_idx])
    
    dist.append(np.array([i, np.mean(curr_bin), len(curr_bin)]))
dist = np.array(dist)

dist_degrees = pd.DataFrame(data={'Bin':dist[:,0], 'BinMean':dist[:,1], 'Count':dist[:,2]})   #add number of nodes in each
dist_degrees.head(10)

,Bin,BinMean,Count
0,0.0,1.0,5.0
1,1.0,1.0,6.0
2,2.0,1.0,6.0
3,3.0,1.0,6.0
4,4.0,1.0,5.0
5,5.0,1.0,6.0
6,6.0,1.0,6.0
7,7.0,1.0,6.0
8,8.0,1.0,5.0
9,9.0,1.0,6.0


In [104]:
dist_degrees.to_csv('./degrees_dist.csv',index=False)
dist_strength.to_csv('./strength_dist.csv',index=False)

In [105]:
# Count of SKUs in orders
SKUs_per_order = df.groupby('Order#')[['SKU']].count()
SKUs_per_order.head(5)

,SKU
Order#,
E10468817,1
E10781350,1
E10979471,1
E11207393,1
E11341877,1


In [106]:
# Get Order Sizes
temp = SKUs_per_order.groupby('SKU')[['SKU']].count().rename(columns={'SKU':'Count'}).reset_index()
order_sizes = temp.rename(columns={'SKU':'# of SKUs'})
order_sizes.head(5)

,# of SKUs,Count
0,1,3168516
1,2,526924
2,3,168242
3,4,76807
4,5,36585


In [107]:
# Get Order Size percentages
order_sizes['%'] = order_sizes['Count']/order_sizes['Count'].sum()
order_sizes['Cum %'] = order_sizes['%'].cumsum()

# Drop Count columns
order_sizes = order_sizes.drop(['Count'], axis=1)
order_sizes.to_csv('./order_sizes.csv',index=False)
order_sizes.head(10)

,# of SKUs,%,Cum %
0,1,0.786396,0.786396
1,2,0.130778,0.917174
2,3,0.041756,0.958930
3,4,0.019063,0.977992
4,5,0.009080,0.987072
5,6,0.004972,0.992044
6,7,0.002764,0.994808
7,8,0.001698,0.996505
8,9,0.001075,0.997581
9,10,0.000702,0.998283


In [19]:
# Correlation Pair Matrix of top 10 selling SKUs
n=50
top_n_skus = df.groupby('SKU')[['sum']].sum().nlargest(n, 'sum')
top_n_skus

top_n_df = pd.merge(df, top_n_skus, how='inner', on=['SKU']).drop(columns=['sum_y']).rename(columns={'sum_x':'sum'})
# a = top_n_df[['Day', 'SKU', 'sum']]

#Missing days? Assume as 0 sales?
daily_top_n_df = pd.pivot_table(top_n_df, values='sum', index=['Day'], columns=['SKU'], aggfunc=np.sum).fillna(0)
daily_top_n_df.index = pd.to_datetime(daily_top_n_df.index)
daily_top_n_df = daily_top_n_df.sort_index()

daily_top_n_df.to_csv('./daily_top_n_skus.csv',index=False)

In [20]:
# Convert correlation matrix to pairwise SKU dataframe
colnames = list(sku_corr_df.columns)
reset_sku_corr_df = sku_corr_df.reset_index()
pair_SKU_corr = pd.melt(reset_sku_corr_df, id_vars=['SKU'], value_vars=colnames, var_name='SKU_2', value_name='corr')
pair_SKU_corr = pair_SKU_corr.rename(columns={'SKU':'SKU_1'})
pair_SKU_corr

NameError: name 'sku_corr_df' is not defined

In [174]:
pair_SKU_corr.to_csv('./sku_pair_corr.csv',index=False)

KeyboardInterrupt: 

In [88]:
# Only include orders that contained more than 1 SKU
grouped = df.groupby('Order#')
# for (name,group) in grouped:
#     print(name)
#     print(group)

a = grouped.filter(lambda x: len(x) > 1)
a

,Day,SKU,Order#,DIVISION,sum
3,12/19/17,3857519,E19179156,FTW,1
49,9/26/17,3895955,E16354792,FTW,1
80,9/7/17,3895956,E16014891,FTW,1
81,9/26/17,3895956,E16354792,FTW,1
91,10/21/17,3895956,E16770349,FTW,1
93,10/28/17,3895956,E16891507,FTW,1
99,11/10/17,3895956,E17106080,FTW,1
118,11/29/17,3895956,E18114459,FTW,1
151,4/9/17,3895957,E13176232,FTW,1
171,9/7/17,3895957,E16014891,FTW,1


In [79]:
df[df['Order#'] == 'E11863102']

,Day,SKU,Order#,DIVISION,sum
9246,1/20/17,4212824,E11863102,FTW,2
9665,1/20/17,4212825,E11863102,FTW,2


In [52]:
a[(a['Order#'] == 'E11863102')]

,Day,SKU,Order#,DIVISION,sum
9246,1/20/17,4212824,E11863102,FTW,2
9665,1/20/17,4212825,E11863102,FTW,2


In [53]:
a[(a['Order#'] == 'E11884053')]

,Day,SKU,Order#,DIVISION,sum


In [54]:
a.shape

(960, 5)